# **Task 1 - Binary Representations**

Notes

Optional part: Calculate how many bits set in code, bits set is number of 1s

### 1. Implement Bitwise Left Rotation (`rotl`)

In [48]:
MASK_32 = 0xFFFFFFFF  # 32-bit mask to prevent overflow

def rotl(x, n=1):
    n %= 32  # Ensures n is within 0-31
    return ((x << n) & MASK_32) | ((x & MASK_32) >> (32 - n))

#### Tests

In [49]:
test_val = 0x12345678
    
result_4 = rotl(test_val, 4)
result_0 = rotl(test_val, 0)

# Function to format binary output with leading zeros
def to_bin_str(val):
    return bin(val)[2:].zfill(32)  # Ensures it's always 32 bits long

print(f"Original:   0x{test_val:X} ({to_bin_str(test_val)})")
print(f"rotl(4):    0x{result_4:X} ({to_bin_str(result_4)})")
print(f"rotl(0):    0x{result_0:X} ({to_bin_str(result_0)})")

Original:   0x12345678 (00010010001101000101011001111000)
rotl(4):    0x23456781 (00100011010001010110011110000001)
rotl(0):    0x12345678 (00010010001101000101011001111000)


### 2. Implement Bitwise Right Rotation (`rotr`)


In [50]:
MASK_32 = 0xFFFFFFFF  # 32-bit mask to prevent overflow

def rotr(x, n=1):
    
    n %= 32  # Ensure n is within 0-31
    return ((x >> n) & MASK_32) | ((x << (32 - n)) & MASK_32)


#### Tests

In [51]:
test_val = 0x12345678 
    
result_4 = rotr(test_val, 4)
result_1 = rotr(test_val, 1)

def to_bin_str(val):
    return bin(val)[2:].zfill(32)

print(f"Original:   0x{test_val:X} ({to_bin_str(test_val)})")
print(f"rotr(4):    0x{result_4:X} ({to_bin_str(result_4)})")
print(f"rotr(1):    0x{result_1:X} ({to_bin_str(result_1)})")

Original:   0x12345678 (00010010001101000101011001111000)
rotr(4):    0x81234567 (10000001001000110100010101100111)
rotr(1):    0x91A2B3C (00001001000110100010101100111100)


### 3. Implement `ch(x, y, z)` Function


In [52]:
# Chooses bits from y where x has bits set to 1 and from z where x has bits set to 0.
def ch(x, y, z):
    return (x & y) ^ (~x & z)


#### Tests

In [53]:
# x is diferent from both y and z
x_val = 0b1010 
y_val = 0b1100
z_val = 0b1111
result = ch(x_val, y_val, z_val)
# prints result in binary, with at least 4 bits (padded with zeros if necessary).
print(f"ch(0b1010, 0b1100, 0b1111) = 0b{result:04b}")

# x is equal to y and diferent from z
x_val = 0b0000
y_val = 0b1100
z_val = 0b1111
result = ch(x_val, y_val, z_val)
# prints result in binary, with at least 4 bits (padded with zeros if necessary).
print(f"ch(0b1010, 0b1100, 0b1111) = 0b{result:04b}")

# x is equal to z and diferent from y
x_val = 0b1111
y_val = 0b1100
z_val = 0b1111
result = ch(x_val, y_val, z_val)
# prints result in binary, with at least 4 bits (padded with zeros if necessary).
print(f"ch(0b1010, 0b1100, 0b1111) = 0b{result:04b}")

ch(0b1010, 0b1100, 0b1111) = 0b1101
ch(0b1010, 0b1100, 0b1111) = 0b1111
ch(0b1010, 0b1100, 0b1111) = 0b1100


### 4. Implement `maj(x, y, z)` Function

In [54]:
# Implement a function that takes a majority vote of the bits in x, y, z.
# Each bit should be 1 if at least two of three inputs have 1 in that position.
def maj(x, y, z):
    return (x & y) ^ (x & z) ^ (y & z)

#### Tests

In [55]:
# x has a mixture of 1s and 0s
x_val = 0b1010 
y_val = 0b1100  
z_val = 0b1111 
result = maj(x_val, y_val, z_val)
# prints result in binary, with at least 4 bits (padded with zeros if necessary).
print(f"maj(0b1010, 0b1100, 0b1111) = 0b{result:04b}")

# x has all 1s
x_val = 0b1111 
y_val = 0b1100  
z_val = 0b1111 
result = maj(x_val, y_val, z_val)
# prints result in binary, with at least 4 bits (padded with zeros if necessary).
print(f"maj(0b1010, 0b1100, 0b1111) = 0b{result:04b}")

# x has all 0s
x_val = 0b0000
y_val = 0b1100
z_val = 0b1111 
result = maj(x_val, y_val, z_val)
# prints result in binary, with at least 4 bits (padded with zeros if necessary).
print(f"maj(0b1010, 0b1100, 0b1111) = 0b{result:04b}")

maj(0b1010, 0b1100, 0b1111) = 0b1110
maj(0b1010, 0b1100, 0b1111) = 0b1111
maj(0b1010, 0b1100, 0b1111) = 0b1100


# **Task 2 - Hash Functions**

### 1. Convert C Hash Function to Python

In [56]:
# Converts a string to a hash value.
def hash_function(s):

    hashValue = 0
    # Hash value updated for each character in the string
    for char in s:
        # ord() gets ASCII value of the character
        hashValue = ord(char) + 31 * hashValue
    # Ensure the hash value is within 0-100
    return hashValue % 101

#### Tests

In [57]:
test_strings = ["hello", "world", "python", "hash", "coding", "umbrella"]

for s in test_strings:
    print(f"hash({s}) = {hash_function(s)}")

hash(hello) = 17
hash(world) = 34
hash(python) = 91
hash(hash) = 15
hash(coding) = 73
hash(umbrella) = 78


### 2. Expanded Hash Function

In [58]:
def hash_function_expanded(s):
    
    hashValue = 0
    print(f"\nHashing string '{s}':")
    print("-" * 77)

    for index, char in enumerate(s):
        ascii_value = ord(char)
        print(f"Step {index + 1}: char = '{char}' with an ASCII value of {ascii_value}, previous hash was {hashValue}")
        print(f"\thash = {ascii_value} + 31 * {hashValue}")
        hashValue = ascii_value + 31 * hashValue
        print(f"\tNew hash value after processing '{char}: {hashValue}")

    final_hash_value = hashValue % 101
    print(f"\nFinal hash value after modulo 101: {final_hash_value}")
    print("-" * 77)

    return final_hash_value

#### Tests

In [59]:
test_strings = ["hello", "world", "python", "hash", "coding", "umbrella", "rust"]

for s in test_strings:
    hash_function_expanded(s)



Hashing string 'hello':
-----------------------------------------------------------------------------
Step 1: char = 'h' with an ASCII value of 104, previous hash was 0
	hash = 104 + 31 * 0
	New hash value after processing 'h: 104
Step 2: char = 'e' with an ASCII value of 101, previous hash was 104
	hash = 101 + 31 * 104
	New hash value after processing 'e: 3325
Step 3: char = 'l' with an ASCII value of 108, previous hash was 3325
	hash = 108 + 31 * 3325
	New hash value after processing 'l: 103183
Step 4: char = 'l' with an ASCII value of 108, previous hash was 103183
	hash = 108 + 31 * 103183
	New hash value after processing 'l: 3198781
Step 5: char = 'o' with an ASCII value of 111, previous hash was 3198781
	hash = 111 + 31 * 3198781
	New hash value after processing 'o: 99162322

Final hash value after modulo 101: 17
-----------------------------------------------------------------------------

Hashing string 'world':
-----------------------------------------------------------------

### 3. Reasons For Using 31 and 101

**Why 31?**

- It is a **prime number**, this is important because it helps reduce the chance of **collisions** when hashing strings by distributing hash values more evenly accross the hash table. 

- A **collision** occurs when two different inputs produce the same hash value. 

- The multiplication using 31 makes the hash values less predictable and spread out over a wider range, minimising **clustering**.

- 31 is also a small number, reducing the likelyhood of integer overflow in C due to it having fixed sized integers (such as 32 or 64-bit). However, since this has been converted to Python this is no longer an issue as it automatically swtiches from fixed-sized intgers to arbitrary-precision integers when needed. It is still used here though to maintain consistancy between languages and to prevent performance slowdown which can still happen with extremely large numbers

- In C, multiplying by 31 can be optimised by compilers as a simple calculation it provides of **(hashval << 5) - hashval** enables the efficient use of bit-shifting and subtraction instead of pure multiplication. Once again, however, this isn't a significant issue in Python as it handles operations like multiplication and bit-shifting at a higher level so there is no significant performance difference between them.

**Why 101?**

- It is also a **prime number**, in this case used in the modulo operation to limit the hash values to be within a specific range, in this case 0 to 100.

- Similar to 31, it being a prime number helps **distribute** the hash values more evenly across this range.

- Using 101 also helps prevent the **clustering** of hash values, this can occur if the modulo base has **common factors** with the data. Examples of these factors are if the data contains patterns divisible by the modulo base. 101 **doesn't** share common factors with most data patterns, helping to greatly negate this risk.

# **Task 3 - SHA256**

## Calculate the SHA256 Padding for a Given File

### 1. Create Temporary File

In [60]:
import tempfile
import os

In [61]:
with tempfile.NamedTemporaryFile(delete=False, mode="wb") as temp_file:
    temp_file.write(b"abc")
    temp_file_path = temp_file.name

#### Tests

In [62]:
print(f"Temporary file created at: {temp_file_path}")

Temporary file created at: C:\Users\melgo\AppData\Local\Temp\tmppodwhq4q


### 2. Read Temporary File

In [63]:
with open(temp_file_path, "rb") as file:
    data = file.read()

#### Tests

In [64]:
bit_string = " ".join(f"{byte:08b}" for byte in data)
print("Binary contents of the file:", "".join(bit_string))

Binary contents of the file: 01100001 01100010 01100011


### 3. Calculate the Original Length of File Data in Bits

In [65]:
num_bytes = len(data) 
original_length_bits = num_bytes * 8

#### Tests

In [66]:
print(f"Original length in bits: {num_bytes} × 8 = {original_length_bits} (Sum of: {bit_string})")

Original length in bits: 3 × 8 = 24 (Sum of: 01100001 01100010 01100011)


### 4. Append 1 Bit (0x80) Onto End of Data

In [67]:
# Marks the end of the file data
padded_message = data + b'\x80'

#### Tests

In [68]:
print(f"After adding 1-bit: {' '.join(f'{b:08b}' for b in padded_message)}")

After adding 1-bit: 01100001 01100010 01100011 10000000


### 5. Calculate Zero Padding

In [69]:
zero_padding_length = (56 - (len(padded_message) % 64)) % 64

padded_message += b'\x00' * zero_padding_length

#### Tests

In [70]:
print(f"After zero padding: {' '.join(f'{b:08b}' for b in padded_message)}")
print(f"Padding length: {zero_padding_length} bytes") 

After zero padding: 01100001 01100010 01100011 10000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000
Padding length: 52 bytes


### 6. Append the Original Message Length to End of Padded Message

In [71]:
padded_message += original_length_bits.to_bytes(8, 'big')

#### Tests

In [72]:
print(f"Final padded message: {' '.join(f'{b:08b}' for b in padded_message)}")

Final padded message: 01100001 01100010 01100011 10000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00011000


### 7. Extract and Display Only the Padding

In [73]:
padding_hex = padded_message[len(data):]

#### Tests

In [74]:
print(f"SHA-256 Padding (Hex): {' '.join(f'{b:02X}' for b in padding_hex)}")

SHA-256 Padding (Hex): 80 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 18


### 8. Delete the Temporary File

#### File Deletion Confirmation Example

In [75]:
print(f"File exists before deletion: {os.path.exists(temp_file_path)}")

try:
    os.remove(temp_file_path)

    # Check if the file still exists
    if os.path.exists(temp_file_path):
        print("\nError: File was NOT deleted!")
    else:
        print("\nTemporary file deleted successfully.")

except FileNotFoundError:
    print("\nWarning: File already deleted or does not exist.")
except Exception as e:
    print(f"\nUnexpected error: {e}")

print(f"\nFile exists after deletion: {os.path.exists(temp_file_path)}")

File exists before deletion: True

Temporary file deleted successfully.

File exists after deletion: False


#### Error Handling Example if file is not found

In [76]:
print(f"File exists before deletion: {os.path.exists(temp_file_path)}")

try:
    os.remove(temp_file_path)

    # Check if the file still exists
    if os.path.exists(temp_file_path):
        print("\nError: File was NOT deleted!")
    else:
        print("\nTemporary file deleted successfully.")

except FileNotFoundError:
    print("\nWarning: File already deleted or does not exist.")
except Exception as e:
    print(f"\nUnexpected error: {e}")

print(f"\nFile exists after deletion: {os.path.exists(temp_file_path)}")

File exists before deletion: False


File exists after deletion: False


# **Task 4 - Prime Numbers**

#### What are Prime Numbers?

- These are numbers that cannot be exactly divided (i.e. without a remainder) by any whole number other than itself and 1.

### Trial Division Algorithm

In [77]:
def is_prime(n):
    if n < 2:
        return False
    if n in (2, 3):
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

In [78]:
def first_n_primes_trial(n):
    primes = []
    num = 2
    while len(primes) < n:
        if is_prime(num):
            primes.append(num)
        num += 1
    return primes

#### Tests


In [79]:
primes_trial = first_n_primes_trial(1000)
print(primes_trial)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997, 1009, 1013, 1019, 1021, 1031, 1033, 1039, 1049, 1051, 1061, 1063, 1069, 1087, 1091, 1093, 1097, 1103, 1109, 1117, 1123, 1129, 1151, 1153, 1163, 1171, 1181, 1187, 1193, 1201, 1213, 1217, 12

#### Explanation

- Trial Division checks if a number is prime by checking its divisibilty by smaller numbers before it.
- First, it eliminates small cases, in this case numbers less than 2 as 0 and 1 are not prime, and even numbers greater than 2 as they are not prime because they are divisible by 2.
- Next, it tests divisibility only up to the square root of the number, because if the number has a factor larger than the square root of the number, it must also have a smaller factor that will already have been found.
- It skips even numbers and only tests divisibilty by odd numbers (after elimenating 2 and 3). This is because if the number is not divisible by 2, it cannot be divisible by any larger even number, and if the number is divisible by 3 it is also not prime.
- If the number is not divisible by any of these, it is prime.
- The drawbacks of this algorithm is that even with optimisation, it is slow for very large numbers because it has to check many individual numbers. For instance, if the number is 1,000,003, all numbers up the square root of it, which is 1000, must be checked.

### Sieve of Atkin Algorithm

In [80]:
def sieve_of_atkin(limit):
    if limit < 2:
        return []

    ## Initialise the sieve and create list of False values that assumes all numbers are not prime
    sieve = [False] * (limit + 1)
    sieve[2] = sieve[3] = True

    ## Use Quadratic Equations to find potential prime numbers
    for x in range(1, int(limit**0.5) + 1):
        for y in range(1, int(limit**0.5) + 1):
            
            ## Formula 1: Checks if number is modulo 12 = 1 or 5
            n = (4 * x * x) + (y * y)
            if n <= limit and (n % 12 == 1 or n % 12 == 5):
                sieve[n] = not sieve[n]

            ## Formula 2: Checks if number is modulo 12 ≡ 7
            n = (3 * x * x) + (y * y)
            if n <= limit and n % 12 == 7:
                sieve[n] = not sieve[n]

            ## Formula 3: Checks if number is modulo 12 ≡ 11
            n = (3 * x * x) - (y * y)
            if x > y and n <= limit and n % 12 == 11:
                sieve[n] = not sieve[n]

    ## Mark all multiples of known primes as non-prime to eliminate false positives
    for num in range(5, int(limit**0.5) + 1):
        if sieve[num]:
            for multiple in range(num * num, limit + 1, num * num):
                sieve[multiple] = False

    return [num for num in range(limit + 1) if sieve[num]]


In [81]:
primes_atkin = sieve_of_atkin(8000)[:1000]
print(primes_atkin)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997, 1009, 1013, 1019, 1021, 1031, 1033, 1039, 1049, 1051, 1061, 1063, 1069, 1087, 1091, 1093, 1097, 1103, 1109, 1117, 1123, 1129, 1151, 1153, 1163, 1171, 1181, 1187, 1193, 1201, 1213, 1217, 12

#### Explanation

- This is an optimised algorithm for finding all primes up to a given limit.
- Instead of checking each number individually, like Trial Division, it uses mathematical quadratic equations to mark numbers that could be prime.
- It only considers numbers as possible primes if they fit specific repeating patterns in modulo 12 arithmetic (i.e. numbers that can satisfy certain properties when divided by 12). These equations help to quickly skip unnecessary checks.
- False positives are then eliminated by removing multiples of known primes.
- When this filtering is completed, the remaining marked numbers are prime.

# **Task 5 - Roots**

In [82]:
import math
import unittest

In [83]:
def get_fractional_bits(value, bits=32):
    """Extracts the first `bits` binary digits of the fractional part of a number."""
    fractional_part = value - math.floor(value) 
    result = 0
    for i in range(bits):
        fractional_part *= 2
        bit = int(fractional_part)
        result = (result << 1) | bit
        fractional_part -= bit
    return result

In [84]:
class TestGetFractionalBits(unittest.TestCase):

    # Test that input of 0.0 returns 0 since there's no fractional part
    def test_zero(self):
        result = get_fractional_bits(0.0)
        print(f"Test Zero: Input = 0.0, Output = {result:032b}")
        self.assertEqual(result, 0)

    # Test that integer input also returns 0, as it has no fractional part
    def test_integer_input(self):
        result = get_fractional_bits(7)
        print(f"Test Integer: Input = 7, Output = {result:032b}")
        self.assertEqual(result, 0)

    # Test with 0.5 which in binary is exactly 0.1, the first bit should be 1 followed by 31 zeros
    def test_half(self):
        result = get_fractional_bits(0.5)
        expected = int('1' + '0'*31, 2)  # Expected 32-bit binary: 1000...0
        print(f"Test Half: Input = 0.5, Output = {result:032b}")
        self.assertEqual(result, expected)

    # Test with an irrational number: sqrt(2) ≈ 1.414 that it's a valid integer and within the 32-bit range
    def test_sqrt_2(self):
        value = math.sqrt(2)
        result = get_fractional_bits(value)
        print(f"Test √2: Input = {value}, Output = {result:032b}")
        self.assertIsInstance(result, int)
        self.assertLess(result, 2**32)

    # Test custom bit length: only extract 8 bits from π ≈ 3.1415 with the result should be between 0 and 255 (2^8 - 1)
    def test_custom_bits(self):
        value = math.pi
        result = get_fractional_bits(value, bits=8)
        print(f"Test π (8 bits): Input = {value}, Output = {result:08b}")
        self.assertGreaterEqual(result, 0)
        self.assertLess(result, 256)

unittest.TextTestRunner(verbosity=2).run(unittest.TestLoader().loadTestsFromTestCase(TestGetFractionalBits))


test_custom_bits (__main__.TestGetFractionalBits.test_custom_bits) ... ok
test_half (__main__.TestGetFractionalBits.test_half) ... ok
test_integer_input (__main__.TestGetFractionalBits.test_integer_input) ... ok
test_sqrt_2 (__main__.TestGetFractionalBits.test_sqrt_2) ... ok
test_zero (__main__.TestGetFractionalBits.test_zero) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.004s

OK


Test π (8 bits): Input = 3.141592653589793, Output = 00100100
Test Half: Input = 0.5, Output = 10000000000000000000000000000000
Test Integer: Input = 7, Output = 00000000000000000000000000000000
Test √2: Input = 1.4142135623730951, Output = 01101010000010011110011001100111
Test Zero: Input = 0.0, Output = 00000000000000000000000000000000


<unittest.runner.TextTestResult run=5 errors=0 failures=0>

In [85]:
# Using the Atkin Sieve from task 4 to generate the first 100 prime numbers
first_100_primes = primes_atkin[:100] 

In [86]:
# Compute and store the 32-bit fractional binary representation of the square roots
sqrt_fractional_bits = []
for prime in first_100_primes:
    sqrt_val = math.sqrt(prime)
    bits = get_fractional_bits(sqrt_val, bits=32)
    sqrt_fractional_bits.append(bits)

In [87]:
# Define the test class
class TestSqrtFractionalBitsExtraction(unittest.TestCase):

    def test_first_100_primes_sqrt_bits(self):
        first_100_primes = primes_trial[:100]

        print("\nStarting validation for 32-bit fractional extraction from square roots of the first 100 prime numbers...\n")

        # Compute the fractional bits
        sqrt_fractional_bits = []
        for prime in first_100_primes:
            sqrt_val = math.sqrt(prime)
            bits = get_fractional_bits(sqrt_val, bits=32)
            sqrt_fractional_bits.append(bits)

        # Test 1: Correct number of results
        self.assertEqual(len(sqrt_fractional_bits), 100)
        print("Test passed: Output list contains 100 entries.")

        # Test 2: All values are valid 32-bit integers
        for bits in sqrt_fractional_bits:
            self.assertIsInstance(bits, int)
            self.assertGreaterEqual(bits, 0)
            self.assertLess(bits, 2**32)
        print("Test passed: All entries are valid 32-bit integers.")

        # Test 3: No None or NaN values
        for bits in sqrt_fractional_bits:
            self.assertIsNotNone(bits)
            self.assertFalse(math.isnan(bits))
        print("Test passed: No None or NaN values in the output.")

        # Test 4: Deterministic output
        for prime in first_100_primes:
            sqrt_val = math.sqrt(prime)
            bits1 = get_fractional_bits(sqrt_val, bits=32)
            bits2 = get_fractional_bits(sqrt_val, bits=32)
            self.assertEqual(bits1, bits2)
        print("Test passed: Results are reproducible (deterministic).")

        print("\nAll tests completed successfully.")

unittest.TextTestRunner(verbosity=2).run(unittest.TestLoader().loadTestsFromTestCase(TestSqrtFractionalBitsExtraction))


test_first_100_primes_sqrt_bits (__main__.TestSqrtFractionalBitsExtraction.test_first_100_primes_sqrt_bits) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.004s

OK



Starting validation for 32-bit fractional extraction from square roots of the first 100 prime numbers...

Test passed: Output list contains 100 entries.
Test passed: All entries are valid 32-bit integers.
Test passed: No None or NaN values in the output.
Test passed: Results are reproducible (deterministic).

All tests completed successfully.


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [92]:
import pandas as pd
import numpy as np

# Group (prime, bits) into rows of 5 pairs
pairs_per_row = 5
pairs = [(str(p), f"{b:032b}") for p, b in zip(first_100_primes, sqrt_fractional_bits)]

grouped_rows = []
for i in range(0, len(pairs), pairs_per_row):
    row = []
    for prime, bits in pairs[i:i + pairs_per_row]:
        row.extend([prime, bits])
    grouped_rows.append(row)

# Repeating column names
columns = []
for _ in range(pairs_per_row):
    columns.extend(["Prime", "Bits"])

df_clean = pd.DataFrame(grouped_rows, columns=columns)

# Apply center alignment to both headers and data using Styler
styled = df_clean.style.set_table_styles(
    [
        {"selector": "th", "props": [("text-align", "center")]},
        {"selector": "td", "props": [("text-align", "center")]}
    ]
)

# Display without index
from IPython.display import display
display(styled.hide(axis="index"))


Prime,Bits,Prime,Bits,Prime,Bits,Prime,Bits,Prime,Bits
2,01101010000010011110011001100111,3,10111011011001111010111010000101,5,00111100011011101111001101110010,7,10100101010011111111010100111010,11,01010001000011100101001001111111
13,10011011000001010110100010001100,17,00011111100000111101100110101011,19,01011011111000001100110100011001,23,11001011101110111001110101011101,29,01100010100110100010100100101010
31,10010001010110010000000101011010,37,00010101001011111110110011011000,41,01100111001100110010011001100111,43,10001110101101000100101010000111,47,11011011000011000010111000001101
53,01000111101101010100100000011101,59,10101110010111111001000101010110,61,11001111011011001000010111010011,67,00101111011100110100011101111101,71,01101101000110000010011011001010
73,10001011010000111101010001010111,79,11100011011000001011010110010110,83,00011100010001010110000000000010,89,01101111000110010110001100110001,97,11011001010011101011111010110001
101,00001100110001001010011000010001,103,00100110000111011100000111110010,107,01011000000101011010011110111110,109,01110000101101111110110101100111,113,10100001010100010011110001101001
127,01000100111110010011011000110101,131,01110010000011011100110111111101,137,10110100011001110011011010011110,139,11001010001100100000101101110101,149,00110100111000001101010000101110
151,01001001110001111101100110111101,157,10000111101010111011100111110010,163,11000100011000111010001011111100,167,11101100001111111100001111110011,173,00100111001001110111111101101101
179,01100001000010111110101111110010,181,01110100001000001011010010011110,191,11010001111111011000101000110011,193,11100100011101110011010110010100,197,00001001001000011001011111110110
199,00011011010100110000110010010101,211,10000110100111010110001101000010,223,11101110111001010010111001001111,227,00010001000001110110011010001001,229,00100001111110111010001101111011


# **Task 6 - Proof of Work**

In [97]:
import hashlib
from collections import defaultdict
import nltk
from nltk.corpus import words

nltk.download('words')

# Load English words from nltk dictionary
english_words = set(words.words())

# Store best matches
max_leading_zeros = 0
best_words = []

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\melgo\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


# **Task 7 - Turing Machines**

# **Task 8 - Computational Complexity**